# HW 2: Deep Bag-of-Words

![words.jpeg](https://cdn-images-1.medium.com/max/1600/0*JpqZhCNsQ_OGaRkB.jpg)

<br>

In this homework, you will be implementing a deep averaging network, detailed in [Deep Unordered Composition  Rivals Syntactic Methods for Text Classification by Iyyer et al. (2015)](https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf) and training it to do sentiment analysis on the Stanford Sentiment Treebank.

<br>

**Please use all of the starter code that is provided, do not make any changes to the data processing, evaluation, and training functions. Only add code were you're asked to.**

<br>

# Part 0: Read Paper!

Read [Deep Unordered Composition  Rivals Syntactic Methods for Text Classification by Iyyer et al. (2015)](https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf).

## Data Processing

Make sure you've downloaded the Stanford Sentiment Treebank that was used in lab. You can find it [here](http://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip).

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.bernoulli import Bernoulli

import re
import random

random.seed(1)
sst_home = 'data/trees'

# Let's do 2-way positive/negative classification instead of 5-way
easy_label_map = {0:0, 1:0, 2:None, 3:1, 4:1}
    # so labels of 0 and 1 in te 5-wayclassificaiton are 0 in the 2-way. 3 and 4 are 1, and 2 is none
    # because we don't have a neautral class. 

PADDING = "<PAD>"
UNKNOWN = "<UNK>"
max_seq_length = 20

def load_sst_data(path):
    data = []
    with open(path) as f:
        for i, line in enumerate(f): 
            example = {}
            example['label'] = easy_label_map[int(line[1])]
            if example['label'] is None:
                continue
            
            # Strip out the parse information and the phrase labels---we don't need those here
            text = re.sub(r'\s*(\(\d)|(\))\s*', '', line)
            example['text'] = text[1:]
            data.append(example)

    random.seed(1)
    random.shuffle(data)
    return data
     
training_set = load_sst_data(sst_home + '/train.txt')
dev_set = load_sst_data(sst_home + '/dev.txt')
test_set = load_sst_data(sst_home + '/test.txt')

<br>

Next, we'll extract the vocabulary from the data, index each token, and finally convert the sentences into lists of indexed tokens. We are also padding and truncating all sentences to be of length=20. (Why? Think about how to handle batching. This is certainly not the only way! This is just simple.)

In [2]:
import collections
import numpy as np

def tokenize(string):
    return string.split()

def build_dictionary(training_datasets):
    """
    Extract vocabulary and build dictionary.
    """  
    word_counter = collections.Counter()
    for i, dataset in enumerate(training_datasets):
        for example in dataset:
            word_counter.update(tokenize(example['text']))
        
    vocabulary = set([word for word in word_counter])
    vocabulary = list(vocabulary)
    vocabulary = [PADDING, UNKNOWN] + vocabulary
        
    word_indices = dict(zip(vocabulary, range(len(vocabulary))))

    return word_indices, len(vocabulary)

def sentences_to_padded_index_sequences(word_indices, datasets):
    """
    Annotate datasets with feature vectors. Adding right-sided padding. 
    """
    for i, dataset in enumerate(datasets):
        for example in dataset:
            example['text_index_sequence'] = torch.zeros(max_seq_length)

            token_sequence = tokenize(example['text'])
            padding = max_seq_length - len(token_sequence)

            for i in range(max_seq_length):
                if i >= len(token_sequence):
                    index = word_indices[PADDING]
                    pass
                else:
                    if token_sequence[i] in word_indices:
                        index = word_indices[token_sequence[i]]
                    else:
                        index = word_indices[UNKNOWN]
                example['text_index_sequence'][i] = index

            example['text_index_sequence'] = example['text_index_sequence'].long().view(1,-1)
            example['label'] = torch.LongTensor([example['label']])


word_to_ix, vocab_size = build_dictionary([training_set])
sentences_to_padded_index_sequences(word_to_ix, [training_set, dev_set, test_set])

In [3]:
print("Size of training dataset:", len(training_set))
print("\nFirst padded and indexified example in training data:\n", training_set[0])

Size of training dataset: 6920

First padded and indexified example in training data:
 {'label': tensor([0]), 'text': 'Yet another entry in the sentimental oh-those-wacky-Brits genre that was ushered in by The Full Monty and is still straining to produce another smash hit .', 'text_index_sequence': tensor([[ 7254, 13864,  7736,  6012, 15076, 11566,  7423, 14032,  3388,  9459,
          5259,  6012,  7643,  2689,  3834, 15978,  4095,  3994, 13291,  3893]])}


<br>
### Batichify data
We're going to be doing mini-batch training. The following code makes data iterators and a batchifying function.

In [4]:
# This is the iterator we'll use during training. 
# It's a generator that gives you one batch at a time.
def data_iter(source, batch_size):
    dataset_size = len(source)
    start = -1 * batch_size
    order = list(range(dataset_size))
    random.shuffle(order)

    while True:
        start += batch_size
        if start > dataset_size - batch_size:
            # Start another epoch.
            start = 0
            random.shuffle(order)   
        batch_indices = order[start:start + batch_size]
        batch = [source[index] for index in batch_indices]
        yield [source[index] for index in batch_indices]

# This is the iterator we use when we're evaluating our model. 
# It gives a list of batches that you can then iterate through.
def eval_iter(source, batch_size):
    batches = []
    dataset_size = len(source)
    start = -1 * batch_size
    order = list(range(dataset_size))
    random.shuffle(order)

    while start < dataset_size - batch_size:
        start += batch_size
        batch_indices = order[start:start + batch_size]
        batch = [source[index] for index in batch_indices]
        if len(batch) == batch_size:
            batches.append(batch)
        else:
            continue
        
    return batches

# The following function gives batches of vectors and labels, 
# these are the inputs to your model and loss function
def get_batch(batch):
    vectors = []
    labels = []
    for dict in batch:
        vectors.append(dict["text_index_sequence"])
        labels.append(dict["label"])
    return vectors, labels


## Evaluation

We'll be looking at accuracy as our evaluation metric.

In [5]:
# This function outputs the accuracy on the dataset, we will use it during training.
def evaluate(model, data_iter):
    model.eval()
    correct = 0
    total = 0
    for i in range(len(data_iter)):
        vectors, labels = get_batch(data_iter[i])
        vectors = torch.stack(vectors).squeeze()
        labels = torch.stack(labels).squeeze()

        output = model(vectors)
        
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
      
    return correct / float(total)

<br>

## Training Loop

The following function trains the model and reports model accuracy on the train and dev set every 5 epochs.

In [6]:
def training_loop(batch_size, num_epochs, model, loss_, optim, training_iter, dev_iter,
                  train_eval_iter, verbose=True):
    step = 0
    epoch = 0
    total_batches = int(len(training_set) / batch_size)
    accuracies = []
    while epoch <= num_epochs:
        model.train()
        vectors, labels = get_batch(next(training_iter)) 
        vectors = torch.stack(vectors).squeeze() # batch_size, seq_len
        labels = torch.stack(labels).squeeze()
        
    
        model.zero_grad()
        
        output = model(vectors)

        lossy = loss_(output, labels)
        lossy.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optim.step()
        

        if step % total_batches == 0:
            model.eval()
            if epoch % 5 == 0:
                train_acc = evaluate(model, train_eval_iter)
                eval_acc = evaluate(model, dev_iter)
                accuracies.append(eval_acc)
                if verbose:
                    print("Epoch %i; Step %i; Loss %f; Train acc: %f; Dev acc %f" 
                          %(epoch, step, lossy.item(),\
                            train_acc, eval_acc))
            epoch += 1
        step += 1
    
    best_dev = max(accuracies)
    print("Best dev accuracy is {}".format(best_dev))
    return best_dev

<br>

# Part 1: Implement DAN (40 points)

Following the [paper](https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf), implement the Deep Averaging Network (DAN).

Implementation details,
- Instead of using \code{tanh} activations however, use \code{ReLU}. 
- Make the number of layers a variable, not a fixed value.
- Make sure to implement word-dropout.

In [7]:
class DAN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size, 
                 batch_size, n_layers, drop_rate):
        super(DAN, self).__init__()
        
        self.embed = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embedding_size = embedding_dim
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
        self.n_layers = n_layers
        self.drop_rate = drop_rate 
        
        self.layer_0 = nn.Linear(embedding_dim, self.hidden_size)
        self.decoder = nn.Linear(self.hidden_size, self.output_size)
        
        hid_layers = []
        for i in range(self.n_layers):
            setattr(self, 'l'+str(i), nn.Linear(self.hidden_size, self.hidden_size))
            hid_layers.append(nn.Linear(self.hidden_size, self.hidden_size))
            hid_layers.append(nn.ReLU())
        
        self.classifier = nn.Sequential(*hid_layers)
        self.classifier.add_module('decoder', self.decoder)
        self.softmax = nn.Softmax(dim=0)
        
        self.init_weights()
    
    
    def forward(self, input):
        text = input
        length = input.shape[1]
        emb = self.embed(text)
        if self.training and self.drop_rate != 0:
            rw = Bernoulli(self.drop_rate).sample((emb.shape[1], ))
            encoded = emb[:, rw==1].mean(dim=1)
        else:
            encoded = emb.mean(dim=1)
        layer_0 = self.layer_0(encoded)
        logits = self.classifier(layer_0)
        return self.softmax(logits)

    def init_hidden(self):
        h0 = torch.zeros(self.batch_size, self.hidden_size)
        return h0
    
    def init_weights(self):
        initrange = 0.1
        lin_layers = [self.layer_0, self.decoder]
        for i in range(1, self.n_layers):
            lin_layers.append(getattr(self, 'l{}'.format(i)))
        em_layer =  [self.embed]
     
        for layer in lin_layers+em_layer:
            layer.weight.data.uniform_(-initrange, initrange)
            if layer in lin_layers:
                layer.bias.data.fill_(0)

### Train the model!

** Please use the hyperparameters and optimizer provided below. Do not make changes here. **

In [9]:
# Hyperparameters 
input_size = vocab_size
num_labels = 2
hidden_dim = 24
embedding_dim = 8
batch_size = 256
num_layers = 2
learning_rate = 0.001
drop_rate = 0.4
num_epochs = 30


# Build and initialize the model
dan = DAN(vocab_size, embedding_dim, hidden_dim, num_labels, batch_size, num_layers, drop_rate)
dan.init_weights()

# Define loss function and optimizer
loss = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(dan.parameters(), lr=learning_rate)

# Build data iterators
training_iter = data_iter(training_set, batch_size)
train_eval_iter = eval_iter(training_set[0:500], batch_size)
dev_iter = eval_iter(dev_set[:500], batch_size)

# Train the model
training_loop(batch_size, num_epochs, dan, loss, optimizer, training_iter, dev_iter, train_eval_iter)

Epoch 0; Step 0; Loss 0.693146; Train acc: 0.503906; Dev acc 0.531250
Epoch 5; Step 135; Loss 0.693138; Train acc: 0.546875; Dev acc 0.585938
Epoch 10; Step 270; Loss 0.689602; Train acc: 0.750000; Dev acc 0.699219
Epoch 15; Step 405; Loss 0.689462; Train acc: 0.753906; Dev acc 0.703125
Epoch 20; Step 540; Loss 0.689611; Train acc: 0.769531; Dev acc 0.714844
Epoch 25; Step 675; Loss 0.689673; Train acc: 0.796875; Dev acc 0.734375
Epoch 30; Step 810; Loss 0.689362; Train acc: 0.808594; Dev acc 0.718750
Best dev accuracy is 0.734375


0.734375

<br>

# Part 2: Hyperparameter tuning (40 points)

Tune the DAN for learning rate, number of layers, and drop-out rate. Select a range for each parameter and then do a random search over these hyperparameters, trying a minimum 5 permutations of hyperparameters. Report results and the best hyperparameters you found. Do you see any patterns in your results?

In [13]:
def training_loop_(batch_size, num_epochs, model, loss_, optim, training_iter, dev_iter,
                  train_eval_iter, verbose=True):
    step = 0
    epoch = 0
    total_batches = int(len(training_set) / batch_size)
    accuracies = []
    while epoch <= num_epochs:
        model.train()
        vectors, labels = get_batch(next(training_iter)) 
        vectors = torch.stack(vectors).squeeze() # batch_size, seq_len
        labels = torch.stack(labels).squeeze()
        
    
        model.zero_grad()
        
        output = model(vectors)

        lossy = loss_(output, labels)
        lossy.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optim.step()
        

        if step % total_batches == 0:
            model.eval()
            if epoch % 5 == 0:
                train_acc = evaluate(model, train_eval_iter)
                eval_acc = evaluate(model, dev_iter)
                accuracies.append(eval_acc)
            epoch += 1
        step += 1
    
    best_dev = max(accuracies)
    print("Best dev accuracy is {}".format(best_dev))
    return best_dev, lossy.item()

In [16]:
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

# Hyperparameters 
input_size = vocab_size
num_labels = 2
hidden_dim = 24
embedding_dim = 8
batch_size = 256
# num_layers = 2
# learning_rate = 0.001
# drop_rate = 0.4
num_epochs = 30

def grid_search(learning_rate, num_layers, drop_rate, vocab_size = input_size,  embedding_dim = 8, hidden_dim = 24, 
                num_labels = 2, batch_size = 256):
    
    # Build and initialize the model
    dan = DAN(vocab_size, embedding_dim, hidden_dim, num_labels, batch_size, num_layers, drop_rate)
    dan.init_weights()

    # Define loss function and optimizer
    loss = nn.CrossEntropyLoss()  
    optimizer = torch.optim.Adam(dan.parameters(), lr=learning_rate)

    # Build data iterators
    training_iter = data_iter(training_set, batch_size)
    train_eval_iter = eval_iter(training_set[0:500], batch_size)
    dev_iter = eval_iter(dev_set[:500], batch_size)

    # Train the model
    best_acc, test_loss =training_loop_(batch_size, num_epochs, dan, loss, optimizer, training_iter, dev_iter, train_eval_iter)
    return best_acc, test_loss
    
hyperp = {'learning_rate' : [0.001, 0.005, 0.01, 0.05, 0.1],  
 'num_layers' : [1, 2, 3, 4, 5, 6], 
 'drop_rate' : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6], 
}

result = {'learning_rate' : [],  
 'num_layers' : [], 
 'drop_rate' : [], 
 'best_acc' : [], 
 'test_loss' : []
}



for lr in hyperp['learning_rate']:
    for n_layer in hyperp['num_layers']:
        for drop in hyperp['drop_rate']:
            print('learning_rate : '+ str(lr) , 'num_layers : '+ str(n_layer), 'drop_rate : '+ str(drop))
            best_acc, test_loss = grid_search(lr, n_layer, drop)
            result['learning_rate'] += [lr]
            result['num_layers']+= [n_layer]
            result['drop_rate']+= [drop]
            result['best_acc']+= [best_acc]
            result['test_loss']+= [test_loss]      

learning_rate : 0.001 num_layers : 1 drop_rate : 0.1
Best dev accuracy is 0.453125
learning_rate : 0.001 num_layers : 1 drop_rate : 0.2
Best dev accuracy is 0.48828125
learning_rate : 0.001 num_layers : 1 drop_rate : 0.3
Best dev accuracy is 0.7578125
learning_rate : 0.001 num_layers : 1 drop_rate : 0.4
Best dev accuracy is 0.79296875
learning_rate : 0.001 num_layers : 1 drop_rate : 0.5
Best dev accuracy is 0.8203125
learning_rate : 0.001 num_layers : 1 drop_rate : 0.6
Best dev accuracy is 0.796875
learning_rate : 0.001 num_layers : 2 drop_rate : 0.1
Best dev accuracy is 0.4609375
learning_rate : 0.001 num_layers : 2 drop_rate : 0.2
Best dev accuracy is 0.546875
learning_rate : 0.001 num_layers : 2 drop_rate : 0.3
Best dev accuracy is 0.7734375
learning_rate : 0.001 num_layers : 2 drop_rate : 0.4
Best dev accuracy is 0.80859375
learning_rate : 0.001 num_layers : 2 drop_rate : 0.5
Best dev accuracy is 0.79296875
learning_rate : 0.001 num_layers : 2 drop_rate : 0.6
Best dev accuracy is 0

Best dev accuracy is 0.64453125
learning_rate : 0.01 num_layers : 5 drop_rate : 0.4
Best dev accuracy is 0.67578125
learning_rate : 0.01 num_layers : 5 drop_rate : 0.5
Best dev accuracy is 0.74609375
learning_rate : 0.01 num_layers : 5 drop_rate : 0.6
Best dev accuracy is 0.71484375
learning_rate : 0.01 num_layers : 6 drop_rate : 0.1
Best dev accuracy is 0.53515625
learning_rate : 0.01 num_layers : 6 drop_rate : 0.2
Best dev accuracy is 0.5390625
learning_rate : 0.01 num_layers : 6 drop_rate : 0.3
Best dev accuracy is 0.625
learning_rate : 0.01 num_layers : 6 drop_rate : 0.4
Best dev accuracy is 0.6171875
learning_rate : 0.01 num_layers : 6 drop_rate : 0.5
Best dev accuracy is 0.73828125
learning_rate : 0.01 num_layers : 6 drop_rate : 0.6
Best dev accuracy is 0.70703125
learning_rate : 0.05 num_layers : 1 drop_rate : 0.1
Best dev accuracy is 0.5234375
learning_rate : 0.05 num_layers : 1 drop_rate : 0.2
Best dev accuracy is 0.6015625
learning_rate : 0.05 num_layers : 1 drop_rate : 0.3
B

In [21]:
# import pandas as pd 
# df = pd.DataFrame(result)
# df.to_csv('grid_search_result')
df = pd.read_csv('grid_search_result')

**Answer**:
1. The best parameter is when the learning_rate = 0.001, num_layers = 1, drop_rate = 0.5. It's more likely to get higher test performance when optimization learning_rate is smaller with a fewer number of layers and higher drop rate. Smaller learning rate means that it takes much smaller steps to reach the local or global minimum. Higher drop rate and fewer layers mean that the model will generalize well with less variance and avoid too much overfitting. And it seems that this model has some overfitting problem, and this is why it still performs well when the drop rate is set to 0.5.

In [16]:
df = df.iloc[:, 1:]
df.sort_values('best_acc', axis=0, ascending=False).head(10)

,learning_rate,num_layers,drop_rate,best_acc,test_loss
4,0.001,1,0.5,0.820312,0.689723
9,0.001,2,0.4,0.808594,0.689347
5,0.001,1,0.6,0.796875,0.689615
3,0.001,1,0.4,0.792969,0.689437
10,0.001,2,0.5,0.792969,0.689325
40,0.005,1,0.5,0.792969,0.689289
89,0.010,3,0.6,0.789062,0.689506
77,0.010,1,0.6,0.785156,0.689283
41,0.005,1,0.6,0.785156,0.689281
21,0.001,4,0.4,0.781250,0.689472


2. You can tell from the table below that a fixed epoch method might not converge a certain paramter settings, e.g. first example in the table where test_loss = NaN. It might be good to set up another converge criterion.  

In [25]:
df.head(10)

,learning_rate,num_layers,drop_rate,best_acc,test_loss
0,0.001,1,0.1,0.453125,NaN
1,0.001,1,0.2,0.488281,NaN
2,0.001,1,0.3,0.757812,0.689484
3,0.001,1,0.4,0.792969,0.689437
4,0.001,1,0.5,0.820312,0.689723
5,0.001,1,0.6,0.796875,0.689615
6,0.001,2,0.1,0.460938,NaN
7,0.001,2,0.2,0.546875,NaN
8,0.001,2,0.3,0.773438,0.689738
9,0.001,2,0.4,0.808594,0.689347


<br>

# Part 3: short questions (20 points)

1. Name, and briefly describe, 3 other possible composition functions, instead of the DAN, you could use to build sentence representations.
2. Explain how dropout regularizes a model.
3. What are the shortcomings for training for a fixed number of epochs? Give an alternative.
4. Explain why you might use random search rather than grid search.

_Bonus (5 points): briefly describe the Nelder–Mead method and how you might use it to do hyperparamter tuning. What are the tradeoffs between using Nelder-Mead vs random search?_

1. Name, and briefly describe, 3 other possible composition functions, instead of the DAN, you could use to build sentence representations.

**Answer**:

Recursive Neural Networks (RecNN) : According to wikipedia, "A recursive neural network is created by applying the same set of weights recursively over a structured input, to produce a structured prediction over variable-size input structures, or a scalar prediction on it, by traversing a given structure in topological order". And it works well with the word combination. "In the case of a RNN we are feeding the hidden layers from the previous step as an additional input into the next step."

Convolutional Neural Networks (CNN) : "CNN is looking for the same patterns over different regions of sentences. CNN take a fixed size input and generate fixed-size outputs  [reference](https://datascience.stackexchange.com/questions/11619/rnn-vs-cnn-at-a-high-level)" "CNN encoder performs convolution and pooling operations on an input sentence, then uses a fully connected layer to produce a fixed-length encoding of the sentence. This encoding vector is then fed into a long short-term memory (LSTM) recurrent network to produce a target sentence. [reference](http://aclweb.org/anthology/D17-1254)"

Recurrent Neural Networks (RNN) : "A recurrent neural network (RNN) is a class of artificial neural network where connections between nodes form a directed graph along a temporal sequence. This allows it to exhibit temporal dynamic behavior. Unlike feedforward neural networks, RNNs can use their internal state (memory) to process sequences of inputs [reference](https://en.wikipedia.org/wiki/Recurrent_neural_network)"

2. Explain how dropout regularizes a model.

**Answer**: 

Dropout: Dropping some units and their connections from the network at random during training. In this case, it tries to decrease the complexity of the model. In this case, it will avoid over fits the training data and decrease bias. 

3. What are the shortcomings for training for a fixed number of epochs? Give an alternative.

**Answer:** 

The shortcomings for a fixed number of epochs is that after a fixed epochs we are not sure about whether the optimization and loss converges or not. Thus, an alternative is that, rather than setting a fixed epochs, we can determine a convergence criterion, such as the optimization steps end when the loss function rarely decreases.  




4. Explain why you might use random search rather than grid search.

**Answer**:

As random values are selected at each instance, it is highly likely that the whole of action space has been reached because of the randomness, which takes a huge amount of time to cover every aspect of the combination during grid search since it uniformly searches the whole space. This works best under the assumption that not all hyperparameters are equally important. In this search pattern, random combinations of parameters are considered in every iteration. The chances of finding the optimal parameter are comparatively higher in random search because of the random search pattern where the model might end up being trained on the optimised parameters without any aliasing. [reference](https://www.analyticsindiamag.com/why-is-random-search-better-than-grid-search-for-machine-learning/)